# Generate Unconditional Samples (GPT-2)

## Install Required Libraries

In [1]:
#Install Required Libraries
!pip3 install -q -r requirements.txt

In [2]:
#Download the models
#Model Options: 117M or 345M
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 485kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 19.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 650kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 66.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 5.05Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 12.9Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 27.9Mit/s]                                                       


## Import Libraries

In [3]:
import json
import os
import numpy as np
import tensorflow as tf

In [4]:
import model, sample, encoder

## Function to Generate Samples

In [5]:
def sample_model(
    model_name='117M',
    seed=None,
    nsamples=0,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',
):
    """
    Run the sample_model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to
     reproduce results
    :nsamples=0 : Number of samples to return, if 0, continues to
     generate samples indefinately.
    :batch_size=1 : Number of batches (only affects speed/memory).
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        np.random.seed(seed)
        tf.set_random_seed(seed)

        output = sample.sample_sequence(
            hparams=hparams, length=length,
            start_token=enc.encoder['<|endoftext|>'],
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )[:, 1:]

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        generated = 0
        while nsamples == 0 or generated < nsamples:
            out = sess.run(output)
            for i in range(batch_size):
                generated += batch_size
                text = enc.decode(out[i])
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                print(text)

## Run Model

In [6]:
sample_model(
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
======================================== SAMPLE 1 ========================================
Podcast

Enough stated, we need to start talking about getting science research funding.

In fall of this year, the National Science Foundation will stand to gain ample number of jobs and revenues when it comes to deeper amounts of academic science research.

Fetish tells, "Of the $500 billion in data sharing funding awarded annually, around $500 billion gets made each year under Title 1R." She claims MIT and rival UCLA – short for American Institute of Technology – were awarded about half of what NIST's Science News Challenge money is.

Now GMB Partners and L'Amy Pali